# LayoutLMv3 Transcript Extraction – Multi-Page Dataset

This notebook trains **LayoutLMv3** to extract **Name**, **GPA**, and **Subject block(s)**  
from multiple annotated transcript images exported from **Label Studio** as a single JSON file.


In [1]:
!pip install -U transformers

# **1) Install dependencies**

In [2]:
!apt-get -y install tesseract-ocr
!pip install -q transformers datasets pillow pytesseract

# 🔥 Clean any conflicting local folders / broken cache
!rm -rf microsoft layoutlmv3-base hf_cache

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [3]:
from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor

processor = LayoutLMv3Processor.from_pretrained(
    "microsoft/layoutlmv3-base",
    apply_ocr=False,
    cache_dir="./hf_cache"
)

model = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    cache_dir="./hf_cache"
)

print("Model loaded!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded!


# **2) Imports and label mapping**

In [4]:
import os
import json
import pytesseract
from PIL import Image
import torch
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification

# BIO label mapping used for training and inference
label2id = {
    "O": 0,
    "B-NAME": 1, "I-NAME": 2,
    "B-GPA": 3,  "I-GPA": 4,
    "B-SUBJECT": 5, "I-SUBJECT": 6,
}
id2label = {v: k for k, v in label2id.items()}


# **3) Upload JSON + ALL transcript images**

In [6]:
from google.colab import files

print("Upload your Label Studio JSON export AND all transcript images (.png/.jpg)")
uploaded = files.upload()

print("\nUploaded files:")
for k in uploaded.keys():
    print(" -", k)

# Find the JSON export
json_files = [f for f in uploaded.keys() if f.lower().endswith(".json")]
assert len(json_files) == 1, "Please upload exactly ONE Label Studio JSON export file."
JSON_PATH = json_files[0]
print("\nUsing JSON file:", JSON_PATH)


Upload your Label Studio JSON export AND all transcript images (.png/.jpg)


Saving project-1-at-2025-12-11-08-16-8c969032.json to project-1-at-2025-12-11-08-16-8c969032.json
Saving e464c806-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_t8x85XR.jpg to e464c806-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_t8x85XR.jpg
Saving ca4f9866-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_zlU0vb4.jpg to ca4f9866-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_zlU0vb4.jpg
Saving c4fbc858-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_u4JchcC.jpg to c4fbc858-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_u4JchcC.jpg
Saving b502901d-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_r9vqDQh.jpg to b502901d-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_r9vqDQh.jpg
Saving 179507a6-Sushanth_Arunachalam_211717104139_1_to_8_Semester_Transcripts-compress_manoEnU.jpg to 179507a6-Sushanth_Arunac

# **4) Load Label Studio data**

In [7]:
with open(JSON_PATH, "r") as f:
    ls_data = json.load(f)

print("Number of annotated pages (tasks):", len(ls_data))
print("Keys in first task:", ls_data[0].keys())


Number of annotated pages (tasks): 9
Keys in first task: dict_keys(['id', 'annotations', 'file_upload', 'drafts', 'predictions', 'data', 'meta', 'created_at', 'updated_at', 'inner_id', 'total_annotations', 'cancelled_annotations', 'total_predictions', 'comment_count', 'unresolved_comment_count', 'last_comment_updated_at', 'project', 'updated_by', 'comment_authors'])


# **5) Helper functions (geometry + one-page builder)**

In [8]:
def is_inside(tbox, rect, pad=10):
    """
    Returns True if the token box's center lies inside the (padded) annotation rectangle.
    tbox: [x1,y1,x2,y2] of word
    rect: [x1,y1,x2,y2] of annotation box
    """
    tx1, ty1, tx2, ty2 = tbox
    rx1, ry1, rx2, ry2 = rect

    # expand annotation rect slightly to be forgiving
    rx1 -= pad
    ry1 -= pad
    rx2 += pad
    ry2 += pad

    cx = (tx1 + tx2) / 2.0
    cy = (ty1 + ty2) / 2.0

    return (rx1 <= cx <= rx2) and (ry1 <= cy <= ry2)


def build_example(item):
    """
    Convert one Label Studio task into:
      - img (PIL)
      - words: list[str]
      - labels: list[str] BIO labels per word
      - norm_boxes: list[[x1,y1,x2,y2]] in 0-1000 LayoutLM space
    """
    # Image filename from LS export
    if "file_upload" in item:
        img_name = os.path.basename(item["file_upload"])
    else:
        # fallback
        img_name = os.path.basename(item["data"]["image"])

    if not os.path.exists(img_name):
        raise FileNotFoundError(f"Image file not found in Colab: {img_name}. Make sure it was uploaded.")

    img = Image.open(img_name).convert("RGB")
    img_w, img_h = img.size

    # 1) Label Studio rectangles -> absolute pixel boxes
    boxes = []
    ann = item["annotations"][0]
    ann_results = ann["result"]
    for r in ann_results:
        if r["type"] != "rectanglelabels":
            continue
        label = r["value"]["rectanglelabels"][0].upper()  # NAME / GPA / SUBJECT

        v = r["value"]
        x_perc, y_perc = v["x"], v["y"]
        w_perc, h_perc = v["width"], v["height"]

        x1 = int((x_perc / 100.0) * img_w)
        y1 = int((y_perc / 100.0) * img_h)
        x2 = int(((x_perc + w_perc) / 100.0) * img_w)
        y2 = int(((y_perc + h_perc) / 100.0) * img_h)

        boxes.append({"label": label, "bbox": [x1, y1, x2, y2]})

    # 2) OCR: words + their bounding boxes
    ocr = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    words = []
    word_boxes = []
    for i in range(len(ocr["text"])):
        text = ocr["text"][i].strip()
        if text == "":
            continue
        x = ocr["left"][i]
        y = ocr["top"][i]
        w = ocr["width"][i]
        h = ocr["height"][i]
        words.append(text)
        word_boxes.append([x, y, x + w, y + h])

    # 3) Assign BIO labels
    labels = ["O"] * len(words)
    for b in boxes:
        rect = b["bbox"]
        label_type = b["label"]  # NAME, GPA, SUBJECT

        indices = []
        for i, tbox in enumerate(word_boxes):
            if is_inside(tbox, rect):
                indices.append(i)

        if indices:
            labels[indices[0]] = f"B-{label_type}"
            for j in indices[1:]:
                labels[j] = f"I-{label_type}"

    # 4) Normalize boxes to 0–1000
    norm_boxes = []
    for x1, y1, x2, y2 in word_boxes:
        norm_boxes.append([
            int(1000 * x1 / img_w),
            int(1000 * y1 / img_h),
            int(1000 * x2 / img_w),
            int(1000 * y2 / img_h),
        ])

    return img, words, labels, norm_boxes


# **6) Quick sanity check on labels for 1st page**

In [9]:
img, words, labels, norm_boxes = build_example(ls_data[0])
print("Unique labels on first page:", sorted(set(labels)))

print("\nSome non-O labels on first page:")
count = 0
for w, l in zip(words, labels):
    if l != "O":
        print(l, "→", w)
        count += 1
        if count >= 25:
            break


Unique labels on first page: ['B-GPA', 'B-NAME', 'B-SUBJECT', 'I-NAME', 'I-SUBJECT', 'O']

Some non-O labels on first page:
B-NAME → A
I-NAME → SUSHANTH
I-NAME → ARUNACHALAM
I-NAME → INSTITUTE
I-NAME → OF
I-NAME → TECHN
B-SUBJECT → 5
I-SUBJECT → 5
B-SUBJECT → si
I-SUBJECT → lc
I-SUBJECT → URS
I-SUBJECT → :
I-SUBJECT → COURSE
I-SUBJECT → TITLE
I-SUBJECT → SEM
I-SUBJECT → oe
I-SUBJECT → —
I-SUBJECT → eS
I-SUBJECT → aaa
I-SUBJECT → pou
I-SUBJECT → and
I-SUBJECT → Number
I-SUBJECT → Theory
I-SUBJECT → CY8ISI
I-SUBJECT → Engineering


# **7) Build LayoutLMv3 encodings for all pages**

In [10]:
processor = LayoutLMv3Processor.from_pretrained(
    "microsoft/layoutlmv3-base",
    apply_ocr=False,
    cache_dir="./hf_cache"   # 👈 force a clean local cache
)

dataset = []

for idx, item in enumerate(ls_data):
    img, words, labels, norm_boxes = build_example(item)

    encoding = processor(
        img,
        words,
        boxes=norm_boxes,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512,
    )

    word_ids = encoding.word_ids(0)
    aligned_labels = []
    for w_id in word_ids:
        if w_id is None:
            aligned_labels.append(-100)
        else:
            aligned_labels.append(label2id[labels[w_id]])

    sample = {
        "input_ids": encoding["input_ids"].squeeze(0),
        "attention_mask": encoding["attention_mask"].squeeze(0),
        "bbox": encoding["bbox"].squeeze(0),
        "pixel_values": encoding["pixel_values"].squeeze(0),
        "labels": torch.tensor(aligned_labels),
        "words": words,  # keep for debug/inference
    }
    dataset.append(sample)

print("✅ Built dataset with", len(dataset), "pages.")


✅ Built dataset with 9 pages.


# **8) Train LayoutLMv3 on all pages**

In [11]:
model = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
    cache_dir="./hf_cache"  # force a fresh local cache folder
)



optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(5):   # increase to 15–20 if needed
    total_loss = 0.0
    for sample in dataset:
        batch = {k: v.unsqueeze(0) for k, v in sample.items() if k != "words"}

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch+1} - avg loss: {avg_loss:.4f}")

model.save_pretrained("layoutlmv3_transcripts_multi")
print("✅ Model saved to layoutlmv3_transcripts_multi")


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1621: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Epoch 1 - avg loss: 0.8157
Epoch 2 - avg loss: 0.4133
Epoch 3 - avg loss: 0.2502
Epoch 4 - avg loss: 0.2019
Epoch 5 - avg loss: 0.1718
✅ Model saved to layoutlmv3_transcripts_multi


# **9) Inference on one page + raw entities**


In [12]:
import math

# 1) Pick a GPA page
test_idx = 0  # or 2 or 4
item = ls_data[test_idx]

# Rebuild example (OCR + boxes)
img, words, labels, norm_boxes = build_example(item)

# 2) Encode for model
encoding_infer = processor(
    img,
    words,
    boxes=norm_boxes,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512,
)

model = LayoutLMv3ForTokenClassification.from_pretrained("layoutlmv3_transcripts_multi")
model.eval()

with torch.no_grad():
    outputs = model(**encoding_infer)
    pred_ids = outputs.logits.argmax(-1)[0].tolist()

# --- 3) Decode tokens into entities (NAME / GPA / SUBJECT) ---

entities = []
current_label = None
current_tokens = []

for token, pid in zip(words, pred_ids):
    label = id2label[pid]

    if label == "O":
        if current_tokens:
            entities.append((current_label, " ".join(current_tokens)))
            current_label = None
            current_tokens = []
        continue

    prefix, ent_type = label.split("-", 1)

    if prefix == "B":
        if current_tokens:
            entities.append((current_label, " ".join(current_tokens)))
        current_label = ent_type
        current_tokens = [token]
    elif prefix == "I" and current_label == ent_type:
        current_tokens.append(token)
    else:
        # fallback
        if current_tokens:
            entities.append((current_label, " ".join(current_tokens)))
        current_label = ent_type
        current_tokens = [token]

if current_tokens:
    entities.append((current_label, " ".join(current_tokens)))

print("Raw entities (from model):")
for etype, text in entities:
    print(etype, "→", text)

# --- 4) Extract Name / GPA / Subjects from model predictions ---

name = None
gpa = None
subjects = []

for etype, text in entities:
    if etype == "NAME" and name is None:
        name = text
    elif etype == "GPA" and gpa is None:
        # Clean up possible spaced digits like "8 . 2 8"
        cleaned = text.replace(" ", "")
        cleaned = cleaned.replace("..", ".")
        gpa = cleaned
    elif etype == "SUBJECT":
        subjects.append(text)

print("\nAfter model decoding:")
print("Name (model):", name)
print("GPA (model):", gpa)
print("Subjects (model):", len(subjects))

# --- 5) FALLBACK: if GPA is still None, read it directly from the GPA bounding box ---

if gpa is None:
    print("\n⚠️ Model did not predict GPA. Using fallback from GPA box + OCR.")

    # Find GPA rectangle in Label Studio annotations
    ann_results = item["annotations"][0]["result"]

    # Recompute word_boxes in pixel coords for this page (for safety)
    # (We re-do minimal OCR for bounding-box based extraction)
    import pytesseract
    from PIL import Image

    # Figure out image name again
    img_name = item.get("file_upload", item["data"]["image"])
    img_name = os.path.basename(img_name)
    img_full = Image.open(img_name).convert("RGB")
    img_w, img_h = img_full.size

    ocr = pytesseract.image_to_data(img_full, output_type=pytesseract.Output.DICT)
    ww = []
    wboxes = []
    for i in range(len(ocr["text"])):
        t = ocr["text"][i].strip()
        if t == "":
            continue
        x = ocr["left"][i]
        y = ocr["top"][i]
        w_ = ocr["width"][i]
        h_ = ocr["height"][i]
        ww.append(t)
        wboxes.append([x, y, x + w_, y + h_])

    # Find GPA box
    gpa_rects = []
    for r in ann_results:
        if r["type"] == "rectanglelabels" and r["value"]["rectanglelabels"][0].upper() == "GPA":
            v = r["value"]
            x1 = int((v["x"] / 100.0) * img_w)
            y1 = int((v["y"] / 100.0) * img_h)
            x2 = int(((v["x"] + v["width"]) / 100.0) * img_w)
            y2 = int(((v["y"] + v["height"]) / 100.0) * img_h)
            gpa_rects.append([x1, y1, x2, y2])

    # Take first GPA rect (if multiple)
    if gpa_rects:
        gx1, gy1, gx2, gy2 = gpa_rects[0]
        # collect tokens whose centers fall inside this GPA rect
        gpa_tokens = []
        for t, box in zip(ww, wboxes):
            x1, y1, x2, y2 = box
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2
            if gx1 <= cx <= gx2 and gy1 <= cy <= gy2:
                gpa_tokens.append(t)

        if gpa_tokens:
            raw_gpa_text = " ".join(gpa_tokens)
            cleaned = raw_gpa_text.replace(" ", "")
            cleaned = cleaned.replace("..", ".")
            gpa = cleaned
            print("GPA (fallback/OCR):", gpa)
        else:
            print("No OCR tokens found inside GPA rect.")
    else:
        print("No GPA rectangle found in annotations.")

# --- 6) Final pretty print ---

print("\nFINAL EXTRACTED FIELDS")
print("Name:", name)
print("GPA:", gpa)
print("Subjects (count):", len(subjects))
for s in subjects:
    print(" -", s)


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1621: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Raw entities (from model):
SUBJECT → ANNA
NAME → 5
NAME → lc | MONTH &
SUBJECT → NOV
SUBJECT → Laboratory 2
SUBJECT → 2018 Cabal
SUBJECT → BS81 ysics ans
SUBJECT → JAN 2018
SUBJECT → 10 APR 2000 Gksiol ney
SUBJECT → Ul APR 201g Internet Programming 3 0 10 APR 2020 BE8255 Basic
SUBJECT → Engin eiiaotal Inteliicence C8251 Programming in
SUBJECT → ecoce 3 | Atl <9 APR 2020 di = 8 APR 208
SUBJECT → Mining: 3
SUBJECT → aail coe Ot Tae EMINTAPR 2019 Internet Programming Laboratory Balle:
SUBJECT → ie 8 | APR 2g Mobile Application
SUBJECT → Apr2020 eee an nie Honisetencs 8 APR 2018 Professional Communication
SUBJECT → 2020 $8261 c oe cae Laboratory 10 APR
SUBJECT → APR 2020 GE8261 Engineering Practices Laboratory 9 APR 2018
SUBJECT → Sis NOV
SUBJECT → 2018 Cryptography and Network Security 3 A+ NOV 2020 CS8391 Date Structures 9
SUBJECT → Management 3 A
SUBJECT → Oriented Programming 7 NOV 2018 Human Computer Interaction 3 a 8 NOV 2020 EC8395
SUBJECT → 2018 Service
SUBJECT → NOV 2020 MA8351 Di

# **10) Extract Name, GPA, Subjects**

In [ ]:
import pytesseract
from PIL import Image
import os

def extract_fields_from_boxes(item):
    """
    Use ONLY Label Studio boxes + OCR to extract:
      - NAME  (from NAME box)
      - GPA   (from GPA box)
      - SUBJECT blocks (from SUBJECT box/es)
    """
    # 1) Open image
    img_name = item.get("file_upload", item["data"]["image"])
    img_name = os.path.basename(img_name)
    img = Image.open(img_name).convert("RGB")
    img_w, img_h = img.size

    # 2) Run OCR once on whole image
    ocr = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    tokens = []
    boxes = []
    for i in range(len(ocr["text"])):
        t = ocr["text"][i].strip()
        if t == "":
            continue
        x = ocr["left"][i]
        y = ocr["top"][i]
        w = ocr["width"][i]
        h = ocr["height"][i]
        tokens.append(t)
        boxes.append([x, y, x+w, y+h])

    # Helper: tokens inside a rectangle
    def tokens_in_rect(rect, pad=5):
        x1, y1, x2, y2 = rect
        x1 -= pad; y1 -= pad
        x2 += pad; y2 += pad
        inside = []
        for t, (bx1, by1, bx2, by2) in zip(tokens, boxes):
            cx = (bx1 + bx2) / 2
            cy = (by1 + by2) / 2
            if x1 <= cx <= x2 and y1 <= cy <= y2:
                inside.append(t)
        return inside

    # 3) Collect rects for NAME / GPA / SUBJECT from LS annotations
    ann_results = item["annotations"][0]["result"]
    name_rects = []
    gpa_rects = []
    subj_rects = []

    for r in ann_results:
        if r["type"] != "rectanglelabels":
            continue
        label = r["value"]["rectanglelabels"][0].upper()
        v = r["value"]
        rx1 = int((v["x"] / 100.0) * img_w)
        ry1 = int((v["y"] / 100.0) * img_h)
        rx2 = int(((v["x"] + v["width"]) / 100.0) * img_w)
        ry2 = int(((v["y"] + v["height"]) / 100.0) * img_h)
        rect = [rx1, ry1, rx2, ry2]

        if label == "NAME":
            name_rects.append(rect)
        elif label == "GPA":
            gpa_rects.append(rect)
        elif label == "SUBJECT":
            subj_rects.append(rect)

    # 4) Extract NAME
    name_text = None
    if name_rects:
        name_tokens = tokens_in_rect(name_rects[0])
        if name_tokens:
            # Heuristic cleanup: drop generic words if present
            raw_name = " ".join(name_tokens)
            # You can fine-tune this for your layout; for now just keep as-is:
            name_text = raw_name

    # 5) Extract GPA
    gpa_text = None
    if gpa_rects:
        gpa_tokens = tokens_in_rect(gpa_rects[0])
        if gpa_tokens:
            raw_gpa = "".join(gpa_tokens)  # remove spaces between digits
            raw_gpa = raw_gpa.replace("..", ".")
            # Optional: keep only something that looks like a number
            import re
            m = re.search(r"\d+(\.\d+)?", raw_gpa)
            if m:
                gpa_text = m.group(0)
            else:
                gpa_text = raw_gpa

    # 6) Extract SUBJECT blocks (one string per SUBJECT rect)
    subject_blocks = []
    for rect in subj_rects:
        subj_tokens = tokens_in_rect(rect)
        if subj_tokens:
            subject_blocks.append(" ".join(subj_tokens))

    return name_text, gpa_text, subject_blocks

# ---- Run this on a specific page (choose a GPA page like 0, 2, or 4) ----

test_idx = 0   # or 0, or 4
item = ls_data[test_idx]

name_box, gpa_box, subject_blocks = extract_fields_from_boxes(item)

print("FINAL FIELDS")
print("Name:", name_box)
print("GPA:", gpa_box)
print("Subject blocks:", len(subject_blocks))
for s in subject_blocks:
    print(" -", s)


FINAL FIELDS
Name: A SUSHANTH ARUNACHALAM OF TECHN
GPA: 8.62
Subject blocks: 2
 - si lc SEM oe aaa CY8ISI Engineering C! emistry 5 ae GEBIS1 Boole Solas a Python Program| GE8152 Bog necting CMDs HSS81S1 Communicative aa : 61 Phynior end Chany aboot BS81 ysics ans oi tov Pi ing Laboratory Gksiol ney $e Mop dct inecring BE8255 Basic Electrical, Electronics and Measurement Engin C8251 Programming in C di = een ae oe aail coe Ot Tae nce ie eee an nie Honisetencs $8261 c oe cae Laboratory GE8261 Engineering Practices Laboratory $8351 nae Principles and System Design CS8391 Date Structures CS8392 Object Oriented Programming EC8395 Communication Engineering MA8351 Discrete Mathematics CS8381 CS8382 CS8383 HS8381 CS8451 = et
 - 5 5 URS : COURSE TITLE — eS pou and Number Theory Air Pollution and Control Engineering Networks Laboratory Object Oriented Analysis and Design Laboratory Microprocessors and Microcontrollers Laboratory Cabal Distributed Systems Internet Programming eiiaotal Inteliicenc

In [ ]:
all_results = []

for idx, item in enumerate(ls_data):
    name_box, gpa_box, subject_blocks = extract_fields_from_boxes(item)

    # Clean name as before
    cleaned_name = None
    if name_box:
        bad_bits = ["A", "NAME", "OF", "THE", "CANDIDATE"]
        tokens = name_box.split()
        cleaned_tokens = [t for t in tokens if t.upper() not in bad_bits]
        cleaned_name = " ".join(cleaned_tokens)

    all_results.append({
        "page": idx,
        "name": cleaned_name,
        "gpa": gpa_box,
        "subjects": subject_blocks,
    })

for r in all_results:
    print(f"\n=== Page {r['page']} ===")
    print("Name:", r["name"])
    print("GPA :", r["gpa"])
    print("Subjects blocks:", len(r["subjects"]))



=== Page 0 ===
Name: SUSHANTH ARUNACHALAM TECHN
GPA : 8.62
Subjects blocks: 2

=== Page 1 ===
Name: SUSHANTH ARUNACHALAM
GPA : None
Subjects blocks: 1

=== Page 2 ===
Name: | SUSHANTH ARUNACHALAM
GPA : 20
Subjects blocks: 1

=== Page 3 ===
Name: None
GPA : None
Subjects blocks: 1

=== Page 4 ===
Name: |’ SUSHANTH ARUNACHALAM
GPA : 8.06
Subjects blocks: 1

=== Page 5 ===
Name: SUSHANTH ARUNACHALAM
GPA : None
Subjects blocks: 1

=== Page 6 ===
Name: SUSHANTH ARUNACHALAM
GPA : None
Subjects blocks: 1

=== Page 7 ===
Name: SUSHANTH ARUNAGHALAM
GPA : None
Subjects blocks: 1

=== Page 8 ===
Name: SUSHANTH ARUNACHALAM
GPA : None
Subjects blocks: 1


In [ ]:
print("=== GPA TOKENS DEBUG ===")
for token, pid in zip(words, pred_ids):
    if id2label[pid] in ["B-GPA", "I-GPA"]:
        print(id2label[pid], "→", token)


=== GPA TOKENS DEBUG ===


In [ ]:
all_labels = []
for item in ls_data:
    img, words, labels, boxes = build_example(item)
    all_labels.extend(labels)

print("Unique labels across ALL pages:", sorted(set(all_labels)))


KeyboardInterrupt: 

In [ ]:
for idx, item in enumerate(ls_data):
    img, words, labels, boxes = build_example(item)
    has_gpa = any(l in ["B-GPA", "I-GPA"] for l in labels)
    print(f"Page {idx}: {'HAS GPA' if has_gpa else 'NO GPA'}")


# **Install dependencies**

In [1]:
!pip install gradio==3.50.2 pdf2image pytesseract Pillow transformers==4.44.0 accelerate

# System packages for pdf2image + tesseract
!apt-get update
!apt-get install -y poppler-utils tesseract-ocr

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

# **Imports + basic OCR / GPA / eligibility logic**

In [1]:
!pip install --upgrade pip setuptools wheel

!pip install "pandas<3.0.0" gradio==3.50.2 pdf2image pytesseract Pillow transformers==4.44.0 accelerate

!apt-get update -y
!apt-get install -y poppler-utils tesseract-ocr


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [2]:
# === Synthetic transcript generator ===

from PIL import Image, ImageDraw, ImageFont
import random
import os

# Some fake names and subjects for synthetic transcripts
SYNTHETIC_NAMES = [
    "JOHN MATTHEWS",
    "ALICE KUMAR",
    "PRIYA SINGH",
    "DAVID LEE",
    "MARIA GARCIA",
    "RAHUL SHARMA",
    "EMILY BROWN",
]

SYNTHETIC_SUBJECTS = [
    "MATHEMATICS",
    "COMPUTER SCIENCE",
    "DATA STRUCTURES",
    "MACHINE LEARNING",
    "PHYSICS",
    "CHEMISTRY",
    "ALGORITHMS",
    "STATISTICS",
]

SYNTHETIC_DIR = "synthetic_transcripts"
os.makedirs(SYNTHETIC_DIR, exist_ok=True)


def synthetic_random_gpa():
    return round(random.uniform(7.0, 9.9), 2)


def synthetic_subject_block():
    n = random.randint(4, 7)
    chosen = random.sample(SYNTHETIC_SUBJECTS, n)
    lines = []
    for i, subj in enumerate(chosen, start=1):
        grade = random.choice(["A", "A+", "B", "B+"])
        lines.append(f"{i:02d}  {subj:25s}  {grade}")
    return lines


def generate_synthetic_transcript(index: int):
    # Create a blank white page
    img = Image.new("RGB", (1200, 1600), "white")
    draw = ImageDraw.Draw(img)

    # Try to load a TTF font; if not available, fall back to default
    try:
        font_title = ImageFont.truetype("arialbd.ttf", 40)
        font_label = ImageFont.truetype("arialbd.ttf", 32)
        font_text = ImageFont.truetype("arial.ttf", 28)
    except:
        font_title = ImageFont.load_default()
        font_label = ImageFont.load_default()
        font_text = ImageFont.load_default()

    # Top title
    y = 80
    draw.text((80, y), "UNIVERSITY OF SYNTHETIC STUDIES", font=font_title, fill="black")
    y += 80
    draw.text((80, y), "OFFICIAL ACADEMIC TRANSCRIPT", font=font_label, fill="black")

    # Name block
    y += 100
    name = random.choice(SYNTHETIC_NAMES)
    draw.text((80, y), "NAME OF THE CANDIDATE:", font=font_label, fill="black")
    y += 50
    draw.text((80, y), name, font=font_text, fill="black")

    # Subject block
    y += 100
    draw.text((80, y), "SUBJECTS AND GRADES", font=font_label, fill="black")
    y += 50
    subject_lines = synthetic_subject_block()
    for line in subject_lines:
        draw.text((80, y), line, font=font_text, fill="black")
        y += 40

    # GPA block
    y += 80
    gpa = synthetic_random_gpa()
    draw.text(
        (80, y),
        f"CUMULATIVE GRADE POINT AVERAGE (CGPA): {gpa:.2f}",
        font=font_label,
        fill="black",
    )

    # Save image
    out_path = os.path.join(SYNTHETIC_DIR, f"synthetic_{index:03d}.png")
    img.save(out_path)

    return out_path, name, gpa


def generate_synthetic_batch(n=20):
    records = []
    for i in range(1, n + 1):
        path, name, gpa = generate_synthetic_transcript(i)
        records.append((path, name, gpa))
    return records


print("Synthetic transcript generator ready. Call generate_synthetic_batch(n) to create data.")


Synthetic transcript generator ready. Call generate_synthetic_batch(n) to create data.


In [ ]:
synthetic_records = generate_synthetic_batch(10)
synthetic_records[:3]


In [3]:
import os
os.environ["GRADIO_SKIP_URL_CHECK"] = "true"  # avoid URL checks

import io
import re
from typing import Dict, Any, Tuple

from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import gradio as gr

# Optional: LayoutLMv3 imports (you will use these later)
from transformers import AutoTokenizer, AutoModelForTokenClassification


# ---- BASIC OCR HELPERS ----

def ocr_image(img: Image.Image) -> str:
    text = pytesseract.image_to_string(img)
    return text


def pdf_to_images(pdf_path: str):
    pages = convert_from_path(pdf_path)
    return pages


def extract_name(ocr_text: str) -> str:
    lines = [l.strip() for l in ocr_text.splitlines() if l.strip()]

    # Look for explicit label
    for i, line in enumerate(lines):
        if "NAME OF THE CANDIDATE" in line.upper():
            if i + 1 < len(lines):
                return clean_name(lines[i + 1])
            return clean_name(line.split(":")[-1])

    # Fallback: first ALL CAPS-ish line with 2–4 tokens
    for line in lines[:20]:
        words = line.split()
        if line.isupper() and 2 <= len(words) <= 4:
            return clean_name(line)

    return "UNKNOWN"


def clean_name(name: str) -> str:
    tokens = name.split()
    if len(tokens) > 2 and len(tokens[-1]) == 1:
        tokens = tokens[:-1]
    cleaned = " ".join(tokens)
    return cleaned.title()


def extract_gpa(ocr_text: str) -> float | None:
    pattern = r"(CGPA|GPA|CUMULATIVE GRADE POINT AVERAGE)[^\d]*([0-9]+\.[0-9]+)"
    matches = re.findall(pattern, ocr_text, flags=re.IGNORECASE)

    if matches:
        try:
            return float(matches[-1][1])
        except ValueError:
            pass

    # Fallback: last floating number in the whole doc
    nums = re.findall(r"\d+\.\d+", ocr_text)
    if nums:
        try:
            return float(nums[-1])
        except ValueError:
            pass

    return None


def evaluate_eligibility(gpa: float | None, threshold: float) -> Tuple[str, str]:
    if gpa is None:
        return (
            "GPA NOT FOUND",
            "The system could not find a GPA in this transcript. Please verify manually."
        )

    if gpa >= threshold:
        return (
            "ELIGIBLE",
            f"The student has a GPA of {gpa:.2f}, which is above the threshold of {threshold:.2f}."
        )
    elif gpa >= threshold - 1.0:
        return (
            "REVIEW RECOMMENDED",
            f"The student has a GPA of {gpa:.2f}, slightly below the threshold of {threshold:.2f}. Manual review recommended."
        )
    else:
        return (
            "NOT ELIGIBLE",
            f"The student has a GPA of {gpa:.2f}, which is below the threshold of {threshold:.2f}."
        )


# **LayoutLMv3 integration stub**

In [4]:
# ==== OPTIONAL: LayoutLMv3 model loading (YOU PLUG THIS IN) ====

# Example: if you have your model in Drive at /content/drive/MyDrive/layoutlmv3-model
# from google.colab import drive
# drive.mount('/content/drive')
# MODEL_DIR = "/content/drive/MyDrive/layoutlmv3-model"

layoutlm_model = None
layoutlm_tokenizer = None

def load_layoutlm_model(model_path: str):
    global layoutlm_model, layoutlm_tokenizer
    layoutlm_tokenizer = AutoTokenizer.from_pretrained(model_path)
    layoutlm_model = AutoModelForTokenClassification.from_pretrained(model_path)


def extract_with_layoutlm(image: Image.Image) -> Dict[str, Any]:
    """
    Placeholder: You can use your LayoutLMv3 pipeline here to predict NAME and GPA.
    For now, we just return None so OCR pipeline is used.
    Return format example:
      {
        "name": "John Doe" or None,
        "gpa": 8.62 or None
      }
    """
    if layoutlm_model is None:
        return {"name": None, "gpa": None}

    # TODO: Put your LayoutLMv3 inference code here.
    # Use OCR tokens + bounding boxes + model to classify tokens as NAME/GPA/etc.

    return {"name": None, "gpa": None}


# **Main processing function for Gradio**

In [5]:
import tempfile

def process_transcript(file, gpa_threshold: float) -> Dict[str, Any]:
    """
    file: uploaded from Gradio (in Colab it's usually a temp file path)
    """
    if file is None:
        raise ValueError("No file provided")

    file_path = file.name if hasattr(file, "name") else file

    lower = file_path.lower()

    # ---- PDF case ----
    if lower.endswith(".pdf"):
        images = pdf_to_images(file_path)
        # Use first page for preview and LayoutLM, but OCR all pages
        preview_image = images[0]
        ocr_text = "\n".join(ocr_image(img) for img in images)

    # ---- Image case ----
    else:
        img = Image.open(file_path)
        preview_image = img
        ocr_text = ocr_image(img)

    # ---- Try LayoutLMv3 first (if you wired it) ----
    lm_result = extract_with_layoutlm(preview_image)
    name_from_model = lm_result.get("name")
    gpa_from_model = lm_result.get("gpa")

    # Fallback to OCR-based rules
    name = name_from_model if name_from_model else extract_name(ocr_text)
    gpa = gpa_from_model if gpa_from_model is not None else extract_gpa(ocr_text)

    eligibility, summary = evaluate_eligibility(gpa, gpa_threshold)

    return {
        "name": name,
        "gpa": gpa,
        "eligibility": eligibility,
        "summary": summary,
        "ocr_text": ocr_text,
        "preview_image": preview_image,
    }


In [6]:
# === Program-specific rules + updated process_transcript ===

PROGRAM_RULES = {
    "General": {
        "min_gpa": 8.0,
        "required_subjects": []
    },
    "Computer Science": {
        "min_gpa": 8.2,
        "required_subjects": ["MATHEMATICS", "COMPUTER SCIENCE"]
    },
    "Artificial Intelligence": {
        "min_gpa": 8.5,
        "required_subjects": ["MATHEMATICS", "MACHINE LEARNING"]
    },
    "Data Science": {
        "min_gpa": 8.3,
        "required_subjects": ["MATHEMATICS", "STATISTICS"]
    },
}


def extract_subject_lines(ocr_text: str) -> list:
    """
    Very simple heuristic: treat lines with both letters and digits as possible subjects.
    This is not perfect, but good enough for eligibility checks.
    """
    lines = [l.strip() for l in ocr_text.splitlines() if l.strip()]
    subject_like = []
    for line in lines:
        if any(c.isalpha() for c in line) and any(c.isdigit() for c in line):
            subject_like.append(line.upper())
    return subject_like


def evaluate_eligibility_program(
    gpa: float | None,
    ocr_text: str,
    program: str
) -> Tuple[str, str]:
    rules = PROGRAM_RULES.get(program, PROGRAM_RULES["General"])
    min_gpa = rules["min_gpa"]
    required_subjects = rules["required_subjects"]

    if gpa is None:
        return (
            "GPA NOT FOUND",
            f"For program '{program}', GPA could not be found. Manual review is required."
        )

    subjects = extract_subject_lines(ocr_text)

    # GPA rule
    if gpa < min_gpa:
        return (
            "NOT ELIGIBLE",
            f"GPA {gpa:.2f} is below the minimum {min_gpa:.2f} "
            f"for the {program} program."
        )

    # Required subjects rule (simple substring check)
    missing = []
    for req in required_subjects:
        if not any(req in s for s in subjects):
            missing.append(req)

    if missing:
        return (
            "REVIEW RECOMMENDED",
            f"GPA {gpa:.2f} meets the threshold for {program}, "
            f"but required subject(s) {', '.join(missing)} were not clearly found. "
            "Manual review is recommended."
        )

    return (
        "ELIGIBLE",
        f"GPA {gpa:.2f} meets the {program} threshold ({min_gpa:.2f}) "
        "and required subjects appear present."
    )

def score_gpa_confidence(ocr_text: str, gpa: float | None) -> str:
    """
    Very simple heuristic confidence:
    - HIGH: GPA found and multiple GPA-related keywords present
    - MEDIUM: GPA found and at least one keyword present
    - LOW: GPA missing or no supporting keywords
    """
    if gpa is None:
        return "LOW"

    text_upper = ocr_text.upper()
    hits = 0
    for kw in ["CGPA", "GPA", "GRADE POINT AVERAGE"]:
        if kw in text_upper:
            hits += 1

    if hits >= 2:
        return "HIGH"
    elif hits == 1:
        return "MEDIUM"
    else:
        return "LOW"


# NEW version of process_transcript that takes 'program' into account
def process_transcript(file, gpa_threshold: float, program: str) -> Dict[str, Any]:
    """
    Overrides the earlier process_transcript.
    Uses OCR + (optional) LayoutLMv3 + program-specific eligibility rules.
    """
    if file is None:
        raise ValueError("No file provided")

    # Gradio in Colab usually passes a tempfile object
    file_path = file.name if hasattr(file, "name") else file

    lower = file_path.lower()

    # ---- PDF case ----
    if lower.endswith(".pdf"):
        images = pdf_to_images(file_path)
        # Use first page for preview and LayoutLM, but OCR all pages
        preview_image = images[0]
        ocr_text = "\n".join(ocr_image(img) for img in images)

    # ---- Image case ----
    else:
        img = Image.open(file_path)
        preview_image = img
        ocr_text = ocr_image(img)

    # ---- Try LayoutLMv3 first (if you wired it) ----
    lm_result = extract_with_layoutlm(preview_image)
    name_from_model = lm_result.get("name")
    gpa_from_model = lm_result.get("gpa")

    # Fallback to OCR-based rules
    name = name_from_model if name_from_model else extract_name(ocr_text)
    gpa = gpa_from_model if gpa_from_model is not None else extract_gpa(ocr_text)

    # NEW: compute GPA confidence
    confidence = score_gpa_confidence(ocr_text, gpa)

    # Use program-specific rules instead of simple threshold only
    eligibility, summary = evaluate_eligibility_program(gpa, ocr_text, program)

    return {
        "name": name,
        "gpa": gpa,
        "eligibility": eligibility,
        "summary": summary,
        "ocr_text": ocr_text,
        "preview_image": preview_image,
        "confidence": confidence,   # NEW FIELD
    }



# **Gradio UI in Colab**

In [7]:
DEFAULT_THRESHOLD = 8.0

def analyze(file, threshold, program):
    if file is None:
        return None, "No file uploaded.", "", "", "", "", ""

    result = process_transcript(file, threshold, program)

    name = result["name"]
    gpa = result["gpa"]
    eligibility = result["eligibility"]
    summary = result["summary"]
    ocr_text = result["ocr_text"]
    preview_image = result["preview_image"]
    confidence = result.get("confidence", "UNKNOWN")

    gpa_display = "Not found" if gpa is None else f"{gpa:.2f}"

    return (
        preview_image,
        f"Name: {name}",
        f"GPA: {gpa_display}",
        f"Confidence: {confidence}",
        f"Eligibility: {eligibility}",
        summary,
        ocr_text,
    )



with gr.Blocks(title="Transcript GPA & Eligibility") as demo:
    gr.Markdown("""
    # Transcript GPA & Eligibility Checker
    Upload a transcript (PDF / Image).
    The system will extract the GPA and tell you whether the student is eligible.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(
                label="Upload Transcript (PDF / PNG / JPG)",
                file_types=[".pdf", ".png", ".jpg", ".jpeg"]
            )
            threshold_input = gr.Slider(
                minimum=0.0,
                maximum=10.0,
                value=DEFAULT_THRESHOLD,
                step=0.1,
                label="GPA Threshold"
            )
            program_input = gr.Dropdown(
            choices=["General", "Computer Science", "Artificial Intelligence", "Data Science"],
            value="General",
            label="Target Program"
            )
            analyze_button = gr.Button("Analyze Transcript")

        with gr.Column(scale=2):
            preview_output = gr.Image(label="Transcript Preview")
            name_output = gr.Textbox(label="Extracted University Name")
            gpa_output = gr.Textbox(label="Extracted GPA")
            confidence_output = gr.Textbox(label="GPA Confidence", interactive=False)
            eligibility_output = gr.Textbox(label="Eligibility Result")
            summary_output = gr.Textbox(label="Summary", lines=3)


    ocr_output = gr.Textbox(
        label="OCR Text (for manual review)",
        lines=12
    )

    analyze_button.click(
        fn=analyze,
        inputs=[file_input, threshold_input, program_input],
        outputs=[
            preview_output,
            name_output,
            gpa_output,
            confidence_output,
            eligibility_output,
            summary_output,
            ocr_output,
        ]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://f26308b049c3a8f1f4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
